This is a assignment to deal with unhealthy conversations. The assignment description website is in
https://ml.auc-computing.nl/assignment3.html. The repository I used for the assignment is https://github.com/conversationai/unhealthy-conversations. There are some files I will use in my model: train.csv, test.csv, val.csv. 

In [34]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action="ignore")

In [35]:
data_dir = Path("corpus/")
train_data= pd.read_csv(data_dir / "train.csv")
test_data = pd.read_csv(data_dir / "test.csv")

I delete 'trusted judgements', 'unit id', 'comment' in the columns_use, and the target_column would be the 'healthy' column.

In [36]:

columns_use = ['antagonise','antagonise:confidence','condescending','condescending:confidence',
              'dismissive','dismissive:confidence','generalisation','generalisation:confidence',
              'generalisation_unfair', 'generalisation_unfair:confidence','hostile',
               'hostile:confidence', 'sarcastic','sarcastic:confidence']
target_column = ['healthy']

X_train = train_data[columns_use]
y_train = train_data[target_column]

X_test = test_data[columns_use]
y_test = test_data[target_column]


Create a function combining each sub-attributes with its confidence score, and apply the function to X_train, and X_test. The way to combine the attributes with confidence socre is find the confidence scoe when the attribute is true. Therefore, if the attribute is 1, the confidence score should be 1*score, and if the attribute is 0, the confidence score should be 1*(1-score). Combining the two calculation, the new column should be 1*score + (1-1)*(1-score) for true, and 0*score + (1-0)*(1-score). After giving a new data to the attributes, we drop the columns of attributes-level.

In [37]:
attributes = ['antagonise', 'condescending', 'dismissive', 'generalisation', 
             'generalisation_unfair', 'hostile', 'sarcastic']
confidence = ['antagonise:confidence','condescending:confidence', 
                         'dismissive:confidence', 'generalisation:confidence', 
                         'generalisation_unfair:confidence', 'hostile:confidence', 
                         'sarcastic:confidence']
    
def preprocess(X):
    for i in range(len(attributes)):
        X[attributes[i]] = X[attributes[i]] * X[confidence[i]] + (1- X[attributes[i]]) * (1-X[confidence[i]])
        
preprocess(X_train)
X_train = X_train.drop(confidence, axis=1)

preprocess(X_test)
X_test = X_test.drop(confidence, axis=1)




**Hard voting**


In [38]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier(n_estimators=100)
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("rf", rnd_clf), ("svc", svm_clf)],
    voting="hard",
)
voting_clf.fit(X_train, y_train)

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9428248587570621
RandomForestClassifier 0.9466666666666667
SVC 0.9419209039548022
VotingClassifier 0.943954802259887


**Soft voting**

In [39]:
soft_svm_clf = SVC(probability=True)

soft_voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("rf", rnd_clf), ("svc", soft_svm_clf)],
    voting="soft",
)
soft_voting_clf.fit(X_train, y_train)

for clf in (log_clf, rnd_clf, svm_clf, soft_voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9428248587570621
RandomForestClassifier 0.9466666666666667
SVC 0.9419209039548022
VotingClassifier 0.9441807909604519
